In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install catboost

In [4]:
# train = pd.read_csv("/content/drive/MyDrive/mango/train.csv")
# test = pd.read_csv("/content/drive/MyDrive/mango/test.csv")
# sample = pd.read_csv("/content/drive/MyDrive/mango/sample_submission.csv")

df_train = pd.read_csv('/content/drive/MyDrive/Datathon/train.csv', sep=';')
df_test = pd.read_csv('/content/drive/MyDrive/Datathon/test.csv', sep=';')

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95339 entries, 0 to 95338
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  95339 non-null  int64  
 1   id_season           95339 non-null  int64  
 2   aggregated_family   95339 non-null  object 
 3   family              95339 non-null  object 
 4   category            95339 non-null  object 
 5   fabric              95339 non-null  object 
 6   color_name          95339 non-null  object 
 7   color_rgb           95339 non-null  object 
 8   image_embedding     95339 non-null  object 
 9   length_type         86830 non-null  object 
 10  silhouette_type     82972 non-null  object 
 11  waist_type          23252 non-null  object 
 12  neck_lapel_type     58874 non-null  object 
 13  sleeve_length_type  57336 non-null  object 
 14  heel_shape_type     0 non-null      float64
 15  toecap_type         0 non-null      float64
 16  wove

In [6]:
# Cuenta los valores nulos por columna y los ordena de mayor a menor
missing_values = df_train.isnull().sum().sort_values(ascending=False)

# Filtra solo las columnas que tienen al menos un valor nulo
#missing_values = missing_values[missing_values > 0]

print(missing_values)

heel_shape_type       95339
toecap_type           95339
knit_structure        78413
waist_type            72087
archetype             39732
sleeve_length_type    38003
neck_lapel_type       36465
woven_structure       32829
silhouette_type       12367
length_type            8509
print_type              100
ID                        0
image_embedding           0
category                  0
color_rgb                 0
color_name                0
fabric                    0
id_season                 0
aggregated_family         0
family                    0
moment                    0
phase_in                  0
phase_out                 0
life_cycle_length         0
num_stores                0
num_sizes                 0
has_plus_sizes            0
price                     0
year                      0
num_week_iso              0
weekly_sales              0
weekly_demand             0
Production                0
dtype: int64


In [7]:
COLUMNAS_VACIAS = [
    'heel_shape_type',
    'toecap_type'
    # Si también quieres eliminar las columnas "Unnamed", añádelas aquí:
    # 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32'
]
df_train = df_train.drop(columns=COLUMNAS_VACIAS, errors='ignore')
df_test = df_test.drop(columns=COLUMNAS_VACIAS, errors='ignore')

In [8]:
object_cols = df_train.select_dtypes(include=['object', 'bool']).columns.tolist()
id_cols = ['ID', 'id_season']

CATEGORICAL_FEATURES = list(set(object_cols + id_cols))

for col in CATEGORICAL_FEATURES:
    # Esto asegura que apliques fillna() a las columnas correctas
    if col in df_train.columns:
        df_train[col] = df_train[col].fillna('MISSING')
    if col in df_test.columns:
        df_test[col] = df_test[col].fillna('MISSING')

In [9]:
# Columnas que sabes que son basura y solo aparecen en el test set
TEST_EXCLUSIVOS = ['Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32']

# Eliminar columnas basura del test set
df_test = df_test.drop(columns=TEST_EXCLUSIVOS, errors='ignore')

In [10]:
print(df_train.columns)
print("")
print(df_test.columns)

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'image_embedding', 'length_type',
       'silhouette_type', 'waist_type', 'neck_lapel_type',
       'sleeve_length_type', 'woven_structure', 'knit_structure', 'print_type',
       'archetype', 'moment', 'phase_in', 'phase_out', 'life_cycle_length',
       'num_stores', 'num_sizes', 'has_plus_sizes', 'price', 'year',
       'num_week_iso', 'weekly_sales', 'weekly_demand', 'Production'],
      dtype='object')

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'image_embedding', 'length_type',
       'silhouette_type', 'waist_type', 'neck_lapel_type',
       'sleeve_length_type', 'woven_structure', 'knit_structure', 'print_type',
       'archetype', 'moment', 'phase_in', 'phase_out', 'life_cycle_length',
       'num_stores', 'num_sizes', 'has_plus_sizes', 'price'],
      dtype='object')


In [11]:
import pandas as pd
import numpy as np

# --- 1. AGREGACIÓN DE RENDIMIENTO HISTÓRICO (SOLO EN df_TRAIN) ---

# Agrupar por ID y calcular estadísticas clave
historical_features = df_train.groupby('ID').agg(
    # Medias de rendimiento (potencial de venta)
    hist_media_demand=pd.NamedAgg(column='weekly_demand', aggfunc='mean'),
    hist_media_sales=pd.NamedAgg(column='weekly_sales', aggfunc='mean'),

    # Desviación Estándar (volatilidad/riesgo)
    hist_std_sales=pd.NamedAgg(column='weekly_sales', aggfunc='std'),

    # Máximo de Demanda (pico de éxito)
    hist_max_demand=pd.NamedAgg(column='weekly_demand', aggfunc='max'),

    # Conteo (longevidad) - Usamos una columna disponible para contar
    hist_num_weeks_active=pd.NamedAgg(column='weekly_demand', aggfunc='count')
).reset_index() # .reset_index() convierte ID de vuelta a columna

# Rellenar los NaN creados por la desviación estándar con 0
historical_features = historical_features.fillna(0)

print(historical_features.head())

# Unir estas nuevas features estáticas a ambos DataFrames
# Nota: Los IDs nuevos en df_test que no estén en df_train recibirán NaN aquí.
df_train = pd.merge(df_train, historical_features, on='ID', how='left')
df_test = pd.merge(df_test, historical_features, on='ID', how='left')


   ID  hist_media_demand  hist_media_sales  hist_std_sales  hist_max_demand  \
0   1          67.166667         66.833333       32.663111              135   
1   2         188.833333        182.083333       47.419709              259   
2   3        3543.944444       3081.777778     1071.132911             5441   
3   4        1375.500000       1257.750000      947.717823             2672   
4   6        1835.500000       1717.750000      385.623706             2303   

   hist_num_weeks_active  
0                     12  
1                     12  
2                     18  
3                      8  
4                      8  


In [12]:
# --- 2. DESCOMPOSICIÓN DE FECHAS (phase_in, phase_out) ---

def extract_date_features(df):
    # Asegurarse de que la columna sea de tipo string antes de fillna (por si hay nulos)
    df['phase_in'] = df['phase_in'].astype(str).fillna('1900-01-01')

    # Convertir a datetime
    df['phase_in_dt'] = pd.to_datetime(df['phase_in'], errors='coerce')

    # Extraer features clave de estacionalidad
    df['launch_month'] = df['phase_in_dt'].dt.month
    df['launch_quarter'] = df['phase_in_dt'].dt.quarter
    df['launch_weekday'] = df['phase_in_dt'].dt.dayofweek

    # Limpiar columnas temporales intermedias (opcional)
    df = df.drop(columns=['phase_in_dt'], errors='ignore')

    return df

df_train = extract_date_features(df_train)
df_test = extract_date_features(df_test)

print("✅ Features de fechas (launch_month, etc.) extraídas en ambos sets.")

✅ Features de fechas (launch_month, etc.) extraídas en ambos sets.


In [13]:
print(df_train.columns)
print("")
print(df_test.columns)

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'image_embedding', 'length_type',
       'silhouette_type', 'waist_type', 'neck_lapel_type',
       'sleeve_length_type', 'woven_structure', 'knit_structure', 'print_type',
       'archetype', 'moment', 'phase_in', 'phase_out', 'life_cycle_length',
       'num_stores', 'num_sizes', 'has_plus_sizes', 'price', 'year',
       'num_week_iso', 'weekly_sales', 'weekly_demand', 'Production',
       'hist_media_demand', 'hist_media_sales', 'hist_std_sales',
       'hist_max_demand', 'hist_num_weeks_active', 'launch_month',
       'launch_quarter', 'launch_weekday'],
      dtype='object')

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'image_embedding', 'length_type',
       'silhouette_type', 'waist_type', 'neck_lapel_type',
       'sleeve_length_type', 'woven_structure', 'knit_structure', 'print_type',
       'arche

In [14]:
# 1. Definir la variable Target (y)
df_train_y = df_train['Production']

# 2. Columnas a excluir de las features (X)
# Excluimos el Target y las variables temporales/históricas que ya hemos transformado o no necesitamos
EXCLUDE_COLS = [
    'weekly_sales', 'weekly_demand', 'Production', 'year', 'num_week_iso'
]

# 3. Definir las Features (X)
df_train_x = df_train.drop(columns=EXCLUDE_COLS, errors='ignore')
df_test_x = df_test.drop(columns=EXCLUDE_COLS, errors='ignore')

In [15]:
print(df_train_x.columns)
print("")
print(df_test_x.columns)
print("")
print(df_test_x.head())

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'image_embedding', 'length_type',
       'silhouette_type', 'waist_type', 'neck_lapel_type',
       'sleeve_length_type', 'woven_structure', 'knit_structure', 'print_type',
       'archetype', 'moment', 'phase_in', 'phase_out', 'life_cycle_length',
       'num_stores', 'num_sizes', 'has_plus_sizes', 'price',
       'hist_media_demand', 'hist_media_sales', 'hist_std_sales',
       'hist_max_demand', 'hist_num_weeks_active', 'launch_month',
       'launch_quarter', 'launch_weekday'],
      dtype='object')

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'image_embedding', 'length_type',
       'silhouette_type', 'waist_type', 'neck_lapel_type',
       'sleeve_length_type', 'woven_structure', 'knit_structure', 'print_type',
       'archetype', 'moment', 'phase_in', 'phase_out', 'life_cycle_length',
       'num_sto

In [16]:
# Bloque de Imputación Final para las Features Numéricas

NUMERIC_FEATURES = df_train_x.select_dtypes(include=['float64', 'int64']).columns.tolist()

for col in NUMERIC_FEATURES:
    # Si la columna existe en df_test, se imputan los NaN con 0
    if col in df_train_x.columns:
        df_train_x[col] = df_train_x[col].fillna(0)
    if col in df_test_x.columns:
        df_test_x[col] = df_test_x[col].fillna(0)

In [17]:
# Cuenta los valores nulos por columna y los ordena de mayor a menor
missing_values = df_test_x.isnull().sum().sort_values(ascending=False)

# Filtra solo las columnas que tienen al menos un valor nulo
#missing_values = missing_values[missing_values > 0]

print(missing_values)

ID                       0
id_season                0
aggregated_family        0
family                   0
category                 0
fabric                   0
color_name               0
color_rgb                0
image_embedding          0
length_type              0
silhouette_type          0
waist_type               0
neck_lapel_type          0
sleeve_length_type       0
woven_structure          0
knit_structure           0
print_type               0
archetype                0
moment                   0
phase_in                 0
phase_out                0
life_cycle_length        0
num_stores               0
num_sizes                0
has_plus_sizes           0
price                    0
hist_media_demand        0
hist_media_sales         0
hist_std_sales           0
hist_max_demand          0
hist_num_weeks_active    0
launch_month             0
launch_quarter           0
launch_weekday           0
dtype: int64


In [18]:
from catboost import Pool

# =========================================================
# 1. Definición de la Lista de Features Categóricas Finales
# =========================================================

# Columnas de tipo texto/booleano
CATEGORICAL_FEATURES = df_train_x.select_dtypes(include=['object', 'bool']).columns.tolist()

# Añadir IDs y columnas de fecha/tiempo que deben ser tratadas como categorías
ID_AND_TIME_COLS = ['ID', 'id_season', 'launch_month', 'launch_quarter', 'launch_weekday', 'life_cycle_length']

# Aseguramos que solo incluimos las columnas que existen y no duplicamos
CATEGORICAL_FEATURES.extend([col for col in ID_AND_TIME_COLS if col in df_train_x.columns and col not in CATEGORICAL_FEATURES])


In [19]:
# Columnas que son categorías pero que Pandas pudo haber dejado como float (ej. 2.0, 1.0)
NUMERICAL_CATEGORICAL_COLS = ['ID', 'id_season', 'launch_month', 'launch_quarter', 'launch_weekday', 'life_cycle_length']

print("Corrigiendo tipos de dato: convirtiendo categorías numéricas a string...")

for col in NUMERICAL_CATEGORICAL_COLS:
    if col in df_train_x.columns:
        # Forzar la conversión a string. Esto convierte 2.0 a "2.0".
        df_train_x[col] = df_train_x[col].astype(str)

# Si df_test_x existe, aplica la misma corrección
if 'df_test_x' in locals():
    for col in NUMERICAL_CATEGORICAL_COLS:
        if col in df_test_x.columns:
            df_test_x[col] = df_test_x[col].astype(str)

print("✅ Tipos de dato corregidos. Vuelve a ejecutar la creación del Pool.")

Corrigiendo tipos de dato: convirtiendo categorías numéricas a string...
✅ Tipos de dato corregidos. Vuelve a ejecutar la creación del Pool.


In [20]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# =========================================================
# PASO 1: PARSEAR EL EMBEDDING DE STRING A MATRIZ NUMÉRICA
# =========================================================

def parse_embedding(df, column_name='image_embedding'):
    """Convierte la columna de embedding (string) en un DataFrame de floats."""

    # 1. Asegurar que la columna es string y rellenar NaN si existen
    # (Aunque no deberían si ya limpiaste categóricas, es un paso de seguridad)
    df[column_name] = df[column_name].astype(str).str.replace('[', '').str.replace(']', '')

    # 2. Dividir la string por coma y crear un nuevo DataFrame
    embedding_df = df[column_name].str.split(',', expand=True).astype(float)

    # 3. Asignar nombres a las columnas (ej. embed_0, embed_1, ...)
    num_components = embedding_df.shape[1]
    embedding_df.columns = [f'embed_{i}' for i in range(num_components)]

    # 4. Reemplazar NaN (si la string estaba vacía) con 0
    embedding_df = embedding_df.fillna(0)

    return embedding_df

# Obtener la matriz numérica de embeddings solo para el set de entrenamiento
X_embed_train = parse_embedding(df_train_x.copy(), 'image_embedding')
X_embed_test = parse_embedding(df_test_x.copy(), 'image_embedding') # Necesario para aplicar el mismo PCA después

print(f"Dimensiones de la matriz de embeddings: {X_embed_train.shape}")
# Ejemplo de output: Dimensiones de la matriz de embeddings: (10000, 50) si tienes 50 dimensiones

Dimensiones de la matriz de embeddings: (95339, 512)


In [21]:
# =========================================================
# PASO 2: ESTANDARIZAR LOS DATOS (CRUCIAL PARA PCA)
# =========================================================

# Inicializar y ajustar el escalador SÓLO en el set de entrenamiento
scaler = StandardScaler()
X_embed_train_scaled = scaler.fit_transform(X_embed_train)
X_embed_test_scaled = scaler.transform(X_embed_test) # Aplicar el mismo escalado al test


In [22]:
# =========================================================
# PASO 3: APLICAR PCA
# =========================================================

# Inicializamos PCA sin límite (None) para analizar toda la varianza
pca = PCA(n_components=None, random_state=42)
pca.fit(X_embed_train_scaled)

# Ahora podemos analizar y decidir K

PCA(random_state=42)

In [23]:
# =========================================================
# PASO 4: CÁLCULO DE LA VARIANZA EXPLICADA ACUMULADA
# =========================================================

# Varianza explicada por cada componente
explained_variance_ratio = pca.explained_variance_ratio_

# Varianza explicada acumulada
cumulative_variance = np.cumsum(explained_variance_ratio)

# Creamos un DataFrame para visualizar
variance_df = pd.DataFrame({
    'Componente': range(1, len(cumulative_variance) + 1),
    'Varianza Acumulada': cumulative_variance
})

# Mostrar los puntos clave (ej. dónde se alcanza el 80%, 90% y 95%)
print("\n--- Varianza Explicada Acumulada ---")
for percentage in [0.80, 0.90, 0.95]:
    # Encontrar el índice de la primera componente que supera el umbral
    num_components = np.argmax(cumulative_variance >= percentage) + 1
    print(f"Para el {int(percentage*100)}% de varianza, se necesitan {num_components} componentes.")


--- Varianza Explicada Acumulada ---
Para el 80% de varianza, se necesitan 61 componentes.
Para el 90% de varianza, se necesitan 119 componentes.
Para el 95% de varianza, se necesitan 193 componentes.


In [24]:
# Definición del número óptimo de componentes
K_OPTIMO = 119

# 1. Re-inicializar PCA con K=119
pca_final = PCA(n_components=K_OPTIMO, random_state=42)

# 2. Ajustar y transformar el set de entrenamiento
# Nota: X_embed_train_scaled debe ser la matriz estandarizada
X_pca_train = pca_final.fit_transform(X_embed_train_scaled)

# 3. Transformar el set de prueba usando el mismo ajuste del entrenamiento
X_pca_test = pca_final.transform(X_embed_test_scaled)

# 4. Crear los DataFrames de Componentes
componentes_retenidas = [f'PC{i}' for i in range(1, K_OPTIMO + 1)]

df_pca_train = pd.DataFrame(data=X_pca_train, columns=componentes_retenidas, index=df_train_x.index)
df_pca_test = pd.DataFrame(data=X_pca_test, columns=componentes_retenidas, index=df_test_x.index)

In [25]:
# 1. Eliminar la columna de embedding original del set de entrenamiento
df_train_x = df_train_x.drop(columns=['image_embedding'], errors='ignore')

# 2. Eliminar la columna de embedding original del set de prueba
df_test_x = df_test_x.drop(columns=['image_embedding'], errors='ignore')

print("Columna 'image_embedding' original eliminada de ambos DataFrames.")

Columna 'image_embedding' original eliminada de ambos DataFrames.


In [26]:
# Unir las 119 componentes principales al DataFrame de entrenamiento
df_train_x = pd.concat([df_train_x, df_pca_train], axis=1)

# Unir las 119 componentes principales al DataFrame de prueba
df_test_x = pd.concat([df_test_x, df_pca_test], axis=1)

print(f"✅ Integración PCA completa.")
print(f"Dimensiones de df_train_x después de la unión: {df_train_x.shape}")
print(f"Dimensiones de df_test_x después de la unión: {df_test_x.shape}")

✅ Integración PCA completa.
Dimensiones de df_train_x después de la unión: (95339, 152)
Dimensiones de df_test_x después de la unión: (2250, 152)


In [27]:
print(df_train_x.columns)
print("")
print(df_test_x.columns)

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'length_type', 'silhouette_type',
       ...
       'PC110', 'PC111', 'PC112', 'PC113', 'PC114', 'PC115', 'PC116', 'PC117',
       'PC118', 'PC119'],
      dtype='object', length=152)

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'length_type', 'silhouette_type',
       ...
       'PC110', 'PC111', 'PC112', 'PC113', 'PC114', 'PC115', 'PC116', 'PC117',
       'PC118', 'PC119'],
      dtype='object', length=152)


In [28]:
!pip install kmodes
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore') # Ocultar advertencias de NumPy/Pandas/Sklearn

In [29]:
# --- 1. Definición de Features para el Clustering ---

# Suponemos que tienes de PC1 a PC119 en df_train_x
pc_cols = [col for col in df_train_x.columns if col.startswith('PC')]

# Features Numéricas (PCA, Precio, Distribución)
NUM_FEATURES = pc_cols + ['price', 'life_cycle_length', 'num_stores', 'num_sizes']

# Features Categóricas (Diseño y Funcionalidad)
CAT_FEATURES = [
    'aggregated_family', 'family', 'category', 'fabric', 'color_name',
    'silhouette_type', 'length_type', 'waist_type', 'sleeve_length_type',
    'archetype', 'moment'
]

# --- 2. Crear una Matriz Limpia para el Clustering ---

# Seleccionamos las features de train (la misma lógica se aplicaría a test)
X_train_cluster = df_train_x[NUM_FEATURES + CAT_FEATURES].copy()

# Rellenar NaNs en categóricas para asegurar consistencia
for col in CAT_FEATURES:
    X_train_cluster[col] = X_train_cluster[col].fillna('MISSING_CAT').astype(str)

# --- 3. Estandarización de Features Numéricas ---

# Se ajusta el escalador SÓLO con los datos de entrenamiento
scaler_cluster = StandardScaler()
X_train_cluster[NUM_FEATURES] = scaler_cluster.fit_transform(X_train_cluster[NUM_FEATURES])

print(f"Features numéricas escaladas: {len(NUM_FEATURES)}")
print(f"Features categóricas: {len(CAT_FEATURES)}")

Features numéricas escaladas: 123
Features categóricas: 11


In [30]:
# --- 4. Preparar Data Array y Localización de Categorías ---

# K-Prototypes necesita un array de numpy
X_train_array = X_train_cluster.values

# Obtener los índices de las columnas categóricas en el array (muy importante!)
cat_cols_index = [X_train_cluster.columns.get_loc(col) for col in CAT_FEATURES]

# --- 5. Inicializar y Entrenar K-Prototypes ---

K_CLUSTERS = 50 # Valor inicial (debe ser optimizado)
SEED = 42

print(f"\nIniciando K-Prototypes con K={K_CLUSTERS}...")

kproto = KPrototypes(n_clusters=K_CLUSTERS, init='Huang', n_init=5, max_iter=20, random_state=SEED, verbose=2)
clusters_train = kproto.fit_predict(X_train_array, categorical=cat_cols_index)

# Guardar las etiquetas de cluster en el DataFrame principal
df_train_x['cluster_id'] = clusters_train

print("✅ Clustering completado. Asignaciones añadidas a df_train_x.")


Iniciando K-Prototypes con K=50...
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 70565, ncost: 10125788.50853053
Run: 1, iteration: 2/20, moves: 31414, ncost: 9822575.925649827
Run: 1, iteration: 3/20, moves: 11699, ncost: 9738011.723441621
Run: 1, iteration: 4/20, moves: 5940, ncost: 9714006.52964412
Run: 1, iteration: 5/20, moves: 3409, ncost: 9700243.79016421
Run: 1, iteration: 6/20, moves: 2381, ncost: 9690754.727608407
Run: 1, iteration: 7/20, moves: 2172, ncost: 9681998.763593791
Run: 1, iteration: 8/20, moves: 1635, ncost: 9678046.011100993
Run: 1, iteration: 9/20, moves: 1689, ncost: 9667534.845495049
Run: 1, iteration: 10/20, moves: 1963, ncost: 9650459.54259754
Run: 1, iteration: 11/20, moves: 1166, ncost: 9646210.944595315
Run: 1, iteration: 12/20, moves: 1338, ncost: 9642990.8570187
Run: 1, iteration: 13/20, moves: 1258, ncost: 9640302.285821734
Run: 1, iteration: 14/20, moves: 936, ncost: 9638534.565389767


In [31]:
# --- 6. Calcular Estadísticas del Mercado por Cluster ---

# DataFrame temporal para el cálculo de estadísticas (usamos el target original)
df_cluster_stats = df_train_x.copy()
df_cluster_stats['target'] = df_train_y # Asumimos df_train_y es el target real o log-transformado

# Calcular las métricas agregadas
cluster_metrics = df_cluster_stats.groupby('cluster_id')['target'].agg(
    cluster_mean_demand='mean',
    cluster_max_demand='max',
    cluster_std_sales='std',
    cluster_count='count'
).reset_index()

# Rellenar NaN en desviación estándar (clusters de tamaño 1)
cluster_metrics['cluster_std_sales'] = cluster_metrics['cluster_std_sales'].fillna(0)

print("\n--- Estadísticas Agregadas (Top 5 Clusters) ---")
print(cluster_metrics.head())


# --- 7. Unir las Features al Set de Entrenamiento y Prueba ---

# Unir al set de entrenamiento
df_train_x = pd.merge(df_train_x, cluster_metrics, on='cluster_id', how='left')

# --- Preparación para Unir al Test ---
# NOTA: Para el test, debes:
# 1. Escalar X_test_cluster usando scaler_cluster.transform().
# 2. Aplicar la asignación de cluster: kproto.predict(X_test_array, categorical=cat_cols_index).
# 3. Añadir el cluster_id al df_test_x.
# 4. Unir las estadísticas: pd.merge(df_test_x, cluster_metrics, on='cluster_id', how='left').

print("\n✅ Features de 'Neighborhood Encoding' añadidas a df_train_x.")


--- Estadísticas Agregadas (Top 5 Clusters) ---
   cluster_id  cluster_mean_demand  cluster_max_demand  cluster_std_sales  \
0           0         16659.091003              114552       20392.485565   
1           1         12422.937644               46695        6988.322534   
2           2         21955.786575              110311       22209.327220   
3           3         20336.325454              180415       22918.717569   
4           4         31267.579932              175377       33678.608929   

   cluster_count  
0           1934  
1            866  
2           2324  
3           3469  
4           2352  

✅ Features de 'Neighborhood Encoding' añadidas a df_train_x.


In [32]:
# 1. Crear la matriz limpia para clustering del set de prueba
X_test_cluster = df_test_x[NUM_FEATURES + CAT_FEATURES].copy()

# 2. Rellenar NaNs en categóricas (usando el mismo valor)
for col in CAT_FEATURES:
    X_test_cluster[col] = X_test_cluster[col].fillna('MISSING_CAT').astype(str)

# 3. Escalar las Features Numéricas
# **IMPORTANTE:** Usar .transform() con el scaler_cluster ajustado en el train.
X_test_cluster[NUM_FEATURES] = scaler_cluster.transform(X_test_cluster[NUM_FEATURES])

In [33]:
# 4. Crear el array de NumPy para la predicción
X_test_array = X_test_cluster.values

# 5. Predecir los cluster_id del set de prueba
# **IMPORTANTE:** Usar .predict() con el modelo kproto ajustado en el train.
clusters_test = kproto.predict(X_test_array, categorical=cat_cols_index)

# 6. Añadir el cluster_id al DataFrame de prueba
df_test_x['cluster_id'] = clusters_test

print("✅ Cluster IDs asignados al set de prueba.")

✅ Cluster IDs asignados al set de prueba.


In [34]:
# 7. Unir las estadísticas de rendimiento del cluster (calculadas con datos de entrenamiento)
df_test_x = pd.merge(df_test_x, cluster_metrics, on='cluster_id', how='left')

# 8. (Opcional pero recomendado) Limpieza final de la columna temporal 'cluster_id'
#    Esto evita que CatBoost la confunda con una feature de orden.
df_train_x = df_train_x.drop(columns=['cluster_id'], errors='ignore')
df_test_x = df_test_x.drop(columns=['cluster_id'], errors='ignore')

print(f"\n✅ Features de 'Neighborhood Encoding' añadidas a df_test_x.")
print(f"Dimensiones de df_test_x final: {df_test_x.shape}")


✅ Features de 'Neighborhood Encoding' añadidas a df_test_x.
Dimensiones de df_test_x final: (2250, 156)


In [46]:
print(df_train_x.columns)
print("")
print(df_test_x.columns)

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'length_type', 'silhouette_type',
       ...
       'PC118', 'PC119', 'cluster_mean_demand', 'cluster_max_demand',
       'cluster_std_sales', 'cluster_count', 'mean_demand_DES',
       'max_demand_DES', 'std_sales_DES', 'count_DES'],
      dtype='object', length=160)

Index(['ID', 'id_season', 'aggregated_family', 'family', 'category', 'fabric',
       'color_name', 'color_rgb', 'length_type', 'silhouette_type',
       ...
       'PC118', 'PC119', 'cluster_mean_demand', 'cluster_max_demand',
       'cluster_std_sales', 'cluster_count', 'mean_demand_DES',
       'max_demand_DES', 'std_sales_DES', 'count_DES'],
      dtype='object', length=160)


In [36]:
from kmodes.kmodes import KModes # <-- CAMBIO 1: Importar KModes
import pandas as pd
import numpy as np

# --- 1. Preparación de Features (Solo Categóricas) ---

# Features Categóricas (Diseño y Funcionalidad)
CAT_FEATURES = [
    'aggregated_family', 'family', 'silhouette_type', 'length_type',
    'color_name', 'archetype', 'moment'
]

# Creamos la Matriz con SOLO las Features Categóricas
# X_train_cluster debe ser re-creado solo con CAT_FEATURES
X_train_cluster = df_train_x[CAT_FEATURES].copy()

# Rellenar NaNs y convertir a string (K-Modes trabaja con strings/objetos)
for col in CAT_FEATURES:
    X_train_cluster[col] = X_train_cluster[col].fillna('MISSING_CAT').astype(str)

print(f"Features categóricas para K-Modes: {len(CAT_FEATURES)}")



Features categóricas para K-Modes: 7


In [37]:
# --- 2. Preparar Data Array y Localización de Categorías ---

# K-Modes necesita un array de numpy
X_train_array = X_train_cluster.values

# En K-Modes, todas las columnas son categóricas por defecto.
# Si el array solo contiene categóricas, no es necesario pasar 'categorical',
# o se puede pasar el índice de todas las columnas: [0, 1, 2, 3, 4, 5, 6]
cat_cols_index = list(range(X_train_array.shape[1])) # Índices de todas las columnas


# --- 3. Inicializar y Entrenar K-Modes ---

K_CLUSTERS = 50
SEED = 42
NOMBRE_ID_CLUSTER = 'cluster_id_des' # Usar un nombre único para el ID del cluster (ej. 'cluster_id_cat')
SUFIJO = '_DES'

print(f"\nIniciando K-Modes con K={K_CLUSTERS}...")

# CAMBIO 2: Usar KModes
kmodes = KModes(n_clusters=K_CLUSTERS, init='Huang', n_init=5, max_iter=20, random_state=SEED, verbose=2)
# Usamos 'cat_cols_index' aunque no es estrictamente necesario si ya es solo categórico
clusters_train = kmodes.fit_predict(X_train_array, categorical=cat_cols_index)

# Guardar las etiquetas de cluster en el DataFrame principal
df_train_x[NOMBRE_ID_CLUSTER] = clusters_train

print(f"✅ K-Modes completado. Asignaciones añadidas en '{NOMBRE_ID_CLUSTER}'.")



Iniciando K-Modes con K=50...
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/20, moves: 27711, cost: 217338.0
Run 1, iteration: 2/20, moves: 4546, cost: 216956.0
Run 1, iteration: 3/20, moves: 114, cost: 216956.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/20, moves: 20369, cost: 226704.0
Run 2, iteration: 2/20, moves: 6988, cost: 224427.0
Run 2, iteration: 3/20, moves: 2822, cost: 223794.0
Run 2, iteration: 4/20, moves: 632, cost: 223794.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/20, moves: 28850, cost: 216457.0
Run 3, iteration: 2/20, moves: 3641, cost: 215989.0
Run 3, iteration: 3/20, moves: 146, cost: 215989.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/20, moves: 26836, cost: 220030.0
Run 4, iteration: 2/20, moves: 5232, cost: 219594.0
Run 4, iteration: 3/20, 

In [38]:
# =========================================================
# 4. CALCULAR Y UNIR MÉTRICAS AGREGADAS CON NOMBRES ÚNICOS
# =========================================================

df_cluster_stats = df_train_x.copy()
df_cluster_stats['target'] = df_train_y

# Calcular las métricas agregadas
cluster_metrics_cat = df_cluster_stats.groupby(NOMBRE_ID_CLUSTER)['target'].agg(
    # Usamos el sufijo _CAT
    **{f'mean_demand{SUFIJO}': 'mean',
       f'max_demand{SUFIJO}': 'max',
       f'std_sales{SUFIJO}': 'std',
       f'count{SUFIJO}': 'count'}
).reset_index()

cluster_metrics_cat[f'std_sales{SUFIJO}'] = cluster_metrics_cat[f'std_sales{SUFIJO}'].fillna(0)

print(f"\n--- Estadísticas Agregadas ({SUFIJO}) ---")
print(cluster_metrics_cat.head())

# Unir al set de entrenamiento
df_train_x = pd.merge(
    df_train_x,
    cluster_metrics_cat,
    on=NOMBRE_ID_CLUSTER,
    how='left'
)

print(f"\n✅ Features de 'Neighborhood Encoding' ({SUFIJO}) añadidas a df_train_x.")


--- Estadísticas Agregadas (_DES) ---
   cluster_id_des  mean_demand_DES  max_demand_DES  std_sales_DES  count_DES
0               0     19200.583919          106730   17575.816800       2562
1               1     64574.439617          242420   47299.220225       4488
2               2     33502.708354          223205   39938.015537       2047
3               3     17342.211293          128099   16330.760201       3294
4               4     30400.547221          138771   20645.453449       3706

✅ Features de 'Neighborhood Encoding' (_DES) añadidas a df_train_x.


In [39]:
# --- A. Preparación de Features (Solo Categóricas) ---

# Crear la matriz limpia para clustering del set de prueba (solo CAT_FEATURES)
X_test_cluster = df_test_x[CAT_FEATURES].copy()

# Rellenar NaNs y convertir a string, usando el mismo manejo de NaNs que en el entrenamiento
for col in CAT_FEATURES:
    X_test_cluster[col] = X_test_cluster[col].fillna('MISSING_CAT').astype(str)

# Crear el array de NumPy para la predicción
X_test_array = X_test_cluster.values


# --- B. Predicción del Cluster ---

# 1. Predecir los cluster_id usando el modelo KModes entrenado
# Nota: La lista 'cat_cols_index' no es necesaria en .predict() si el array es 100% categórico
clusters_test = kmodes.predict(X_test_array)

# 2. Asignar el nuevo cluster ID a df_test_x
df_test_x[NOMBRE_ID_CLUSTER] = clusters_test

print(f"✅ Cluster IDs K-Modes asignados a df_test_x en columna '{NOMBRE_ID_CLUSTER}'.")

✅ Cluster IDs K-Modes asignados a df_test_x en columna 'cluster_id_des'.


In [40]:
# --- C. Unión de las Métricas Agregadas ---

# Renombrar la columna del cluster ID en las métricas para que coincida con df_test_x
# (Asumimos que cluster_metrics_cat todavía tiene el nombre 'cluster_id_cat' de la sección 4)
cluster_metrics_cat = cluster_metrics_cat.rename(
    columns={f'cluster_id{SUFIJO}': NOMBRE_ID_CLUSTER}
)


# 3. Unir las métricas de rendimiento del cluster (calculadas con datos de entrenamiento)
df_test_x = pd.merge(
    df_test_x,
    cluster_metrics_cat,
    on=NOMBRE_ID_CLUSTER,
    how='left'
)

# --- D. Limpieza Final ---

# 4. Eliminar el ID del cluster después de la unión para evitar que sea usado como feature ordinal
df_test_x = df_test_x.drop(columns=[NOMBRE_ID_CLUSTER], errors='ignore')
df_train_x = df_train_x.drop(columns=[NOMBRE_ID_CLUSTER], errors='ignore')

print(f"\n✅ Features de 'Neighborhood Encoding' ({SUFIJO}) añadidas a df_test_x.")
print(f"Dimensiones de df_test_x final: {df_test_x.shape}")


✅ Features de 'Neighborhood Encoding' (_DES) añadidas a df_test_x.
Dimensiones de df_test_x final: (2250, 160)


In [45]:
if 'image_embedding' in CATEGORICAL_FEATURES:
    CATEGORICAL_FEATURES.remove('image_embedding')

train_pool = Pool(
    data=df_train_x,
    label=df_train_y, # Usar el target original, o df_train_y_log si se aplica la transformación
    cat_features=CATEGORICAL_FEATURES,
    thread_count=-1 # Para usar todos los cores disponibles
)

In [47]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
     iterations=2500,
     learning_rate=0.05,
     depth=8,
     loss_function="RMSE",
     eval_metric="RMSE",
     random_seed=42,
     # Añadir Detención Temprana (Early Stopping)
     od_type="Iter",  # Tipo de detención: por iteración
     od_wait=300,     # Esperar 300 iteraciones sin mejora antes de parar
     # Otros
     verbose=100
)

model.fit(
     train_pool,  # El Pool de entrenamiento
     verbose=100
)

0:	learn: 33310.8143942	total: 1.55s	remaining: 1h 4m 35s
100:	learn: 6407.0096420	total: 1m 40s	remaining: 39m 48s
200:	learn: 4886.4748329	total: 3m 22s	remaining: 38m 32s
300:	learn: 3872.2806920	total: 5m 6s	remaining: 37m 20s
400:	learn: 3215.1021847	total: 7m 21s	remaining: 38m 31s
500:	learn: 2741.1905043	total: 9m 4s	remaining: 36m 10s
600:	learn: 2381.3999027	total: 10m 47s	remaining: 34m 4s
700:	learn: 2079.5269837	total: 12m 32s	remaining: 32m 10s
800:	learn: 1843.2142964	total: 14m 17s	remaining: 30m 18s
900:	learn: 1649.9964657	total: 16m 2s	remaining: 28m 28s
1000:	learn: 1486.4062007	total: 17m 48s	remaining: 26m 39s
1100:	learn: 1347.5132573	total: 19m 31s	remaining: 24m 48s
1200:	learn: 1230.5162432	total: 21m 16s	remaining: 23m
1300:	learn: 1131.7190062	total: 22m 59s	remaining: 21m 11s
1400:	learn: 1045.1184759	total: 24m 42s	remaining: 19m 23s
1500:	learn: 967.9210182	total: 26m 26s	remaining: 17m 35s
1600:	learn: 898.3293738	total: 28m 12s	remaining: 15m 50s
1700:	

In [48]:
test_prod = df_test_x.copy()
test_predictions = model.predict(df_test_x)

In [49]:
submission = pd.DataFrame({
    "ID": test_prod["ID"],
    "Production": test_predictions
})

submission.to_csv("submission.csv", index=False)

LightBoost

In [50]:
import lightgbm as lgb
import numpy as np
import pandas as pd


CATEGORICAL_COLS = df_train_x.select_dtypes(include=['object', 'category']).columns.tolist()
BEST_ITERATION = 500

params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "verbose": 2,
    "seed": 42,
}

# 1. Asegúrate de que las columnas sean del tipo 'category' en Pandas
for col in CATEGORICAL_COLS:
    df_train_x[col] = df_train_x[col].astype("category")
    df_test_x[col] = df_test_x[col].astype("category")


# 2. Pasar la lista de columnas categóricas a lgb.Dataset
lgb_full = lgb.Dataset(
    df_train_x,
    label=df_train_y,
    categorical_feature=CATEGORICAL_COLS, # <-- Se pasa la lista de los nombres de columnas
    free_raw_data=False
)

print("[FINAL] Entrenando modelo LightGBM...")
model_full = lgb.train(
    params,
    lgb_full,
    num_boost_round=BEST_ITERATION # Usar BEST_ITERATION
)

print("[FINAL] Prediciendo sobre test...")
preds = model_full.predict(
    df_test_x,
    num_iteration=BEST_ITERATION # Usar BEST_ITERATION
)
preds = np.maximum(preds, 0) # evitar negativos

# Nota: Asegúrate de que 'sample_submission.csv' es el archivo de la estructura, no el test.csv
sample_sub = pd.read_csv('/content/drive/MyDrive/Datathon/sample_submission.csv')
submission = sample_sub.copy()
submission["Production"] = preds

print("✅ Predicción final generada.")

[FINAL] Entrenando modelo LightGBM...
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.804072
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.039242
[LightGBM] [Debug] init for col-wise cost 0.008598 seconds, init for row-wise cost 0.147893 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.198525 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42535
[LightGBM] [Info] Number of data points in the train set: 95339, number of used features: 160
[LightGBM] [Info] Start training from score 28927.421055
[LightGBM] [Debug] Re-bagging, using 76157 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[Lig

In [51]:
test_predictions = submission["Production"]
test_prod = submission["ID"]

In [53]:
import lightgbm as lgb
import numpy as np
import pandas as pd
# ... (Tu código de entrenamiento y predicción) ...

# -------------------------------------------------------------
# Zona de la Predicción Final (Tu código original)
# -------------------------------------------------------------
# ... (Tu código de predicción, que calcula 'preds') ...

# Nota: Asegúrate de que 'sample_submission.csv' es el archivo de la estructura, no el test.csv
sample_sub = pd.read_csv('/content/drive/MyDrive/Datathon/sample_submission.csv')
submission = sample_sub.copy()
submission["Production"] = preds # El DataFrame 'submission' ya tiene las columnas "ID" y "Production"

print("✅ Predicción final generada.")

# --- INICIO DEL CÓDIGO PROBLEMÁTICO Y CORRECCIÓN ---

# [ELIMINAR ESTAS LÍNEAS O DEJAR LA SUBMISSION DIRECTA]
# test_predictions = submission["Production"]
# test_prod = submission["ID"]
# submission = pd.DataFrame({
#     "ID": test_prod["ID"],
#     "Production": test_predictions
# })
# [FIN DEL CÓDIGO PROBLEMATICO]

# La variable 'submission' en este punto ya tiene el formato correcto:
# submission.head() mostrará columnas ID y Production

submission.to_csv("submission_light.csv", index=False)
print("✅ Submission guardada correctamente.")

✅ Predicción final generada.
✅ Submission guardada correctamente.
